In [3]:
import pandas as pd

## Want to avoid transforming data as much as possible

- Don't need to format transmission lines from digital atlas (https://digital.atlas.gov.au/datasets/digitalatlas::electricity-transmission-lines/about)
- Data originally comes from Geoscience Australia (https://ecat.ga.gov.au/geonetwork/srv/api/records/fd46e23f-484d-4b40-8f10-9de3ed57fb1f)


In [1]:
pwd()

'/Users/abond/pypsa-au/scripts'

#### Buses

This will include all substations + need to figure out if the other bus ids are from location matching - ask Iman.

Rank by spatial confidence first.

In [6]:
df = pd.read_csv("../data/raw/Transmission_Substations.csv")

# columns = X,Y,objectid,featuretype,class,name,operationalstatus,
#           state,spatialconfidence,revised,ga_guid,description,voltagekv,
#           locality,comment_,x_coordinate,y_coordinate

df.sort_values('spatialconfidence',ascending=False,inplace=True)

# set(df.operationalstatus.to_list()) -> 'Non-Operational', 'Operational'
df = df[df.operationalstatus=='Operational']

df['xy'] = df['x_coordinate'].astype(str) + "," + df['y_coordinate'].astype(str)

xymap = pd.Series(df.xy.values,index=df.name).to_dict()

# xymap

# df.head()
# df

df.to_csv("../data/grid/buses.csv") # saved as buses.

#### Transmission

Grab the busid from ga_guids or objectid, voltage from capacitykv. 

Make df of index,bus0,bus1 extracted from name; split on 'to'. Map index to bus0, apply to bus1. Find locations of all busses, generators close to each bus are given that bus_id (make a coordinates column). Make a column with a REZ_tag to later be able to agg easily.

In [8]:
df = pd.read_csv("../data/raw/Electricity_Transmission_Lines.csv")

# columns = length_m,objectid,featuretype,description,class,name,operationalstatus,
#           state,spatialconfidence,revised,ga_guid,capacitykv,comment_,st_lengthshape

# WARNING: ammended dataset! Coff harbour -> Coffs harbour to improve matching.

df = df.join(df['name'].str.split(' to ', n=1, expand=True).rename(columns={0:'bus_0', 1:'bus_1'}))

# xymap obtained from Transmission_Substations
df["location_0"] = df["bus_0"].map(xymap)
df["location_1"] = df["bus_1"].map(xymap)

df

# df.columns = ['line_id','bus0','bus1','voltage','circuits','length','underground','under_construction','tags','geometry']

df.to_csv("../data/grid/lines.csv") # saved as lines.


In [29]:
# quickly find duplicated values (keep=False lists all)
df[df.name.duplicated(keep=False)]

,length_m,objectid,featuretype,description,class,name,operationalstatus,state,spatialconfidence,revised,ga_guid,capacitykv,comment_,st_lengthshape,bus_0,bus_1,location_0,location_1
56,11207.381220,57,Transmission Line,A network of wires and insulators used to conn...,Overhead,Coffs harbour to Boambee North,Operational,New South Wales,5,1.710370e+12,{258314F3-BA54-41FC-8A81-2414B135F614},66,NaN,0.107856,Coffs harbour,Boambee North,NaN,NaN
57,9947.226289,58,Transmission Line,A network of wires and insulators used to conn...,Overhead,Coffs harbour to Boambee North,Operational,New South Wales,5,1.710370e+12,{8AE2D69F-4287-4ACC-B5BD-7E2B0758BB8D},66,NaN,0.096466,Coffs harbour,Boambee North,NaN,NaN
77,59167.842080,78,Transmission Line,A network of wires and insulators used to conn...,Overhead,Dubbo North to Gilgandra,Operational,New South Wales,5,1.710370e+12,{35993012-AFBB-4537-AAD1-E6AC0FE4A0AE},66,NaN,0.540635,Dubbo North,Gilgandra,"148.63486103,-32.21171185",NaN
1380,2523.146019,1381,Transmission Line,A network of wires and insulators used to conn...,Underground,East Perth Terminal to Wellington Street,Operational,Western Australia,3,1.341880e+12,{447E745D-A8B8-4875-8272-31498091062C},66,NaN,0.025048,East Perth Terminal,Wellington Street,"115.87899148,-31.94564661","115.86494371,-31.95291911"
1401,2472.269756,1402,Transmission Line,A network of wires and insulators used to conn...,Overhead,Snell Street to Casuarina,Operational,Northern Territory,1,1.594680e+12,{20FB0FD8-83FC-498F-A26B-CCEE9855FA7F},33,NaN,0.022529,Snell Street,Casuarina,"130.85797998,-12.42980754","130.87605082,-12.38323148"
1402,7283.924897,1403,Transmission Line,A network of wires and insulators used to conn...,Overhead,Snell Street to Casuarina,Operational,Northern Territory,5,1.341880e+12,{B154A40A-7B27-4398-8289-95F8B666A91F},66,NaN,0.066245,Snell Street,Casuarina,"130.85797998,-12.42980754","130.87605082,-12.38323148"
2481,2523.146019,2482,Transmission Line,A network of wires and insulators used to conn...,Underground,East Perth Terminal to Wellington Street,Non-Operational,Western Australia,3,1.341880e+12,{8F6F0A1B-26E7-4934-8E50-80FEE81C9001},66,NaN,0.025048,East Perth Terminal,Wellington Street,"115.87899148,-31.94564661","115.86494371,-31.95291911"
2491,57216.848120,2492,Transmission Line,A network of wires and insulators used to conn...,Overhead,Dubbo North to Gilgandra,Operational,New South Wales,5,1.710370e+12,{6881BC55-7ED0-4017-AC77-4C4CD0C778B8},66,NaN,0.520465,Dubbo North,Gilgandra,"148.63486103,-32.21171185",NaN


#### Generators

Want: generator_id,bus_id,technology,capacity,tags,geometry,name,under_construction

Find bus_id using location matching and buses.csv?

Extract id, capacity, names etc from Major Power / cross check with GenInfo.

In [11]:
df = pd.read_csv("../data/raw/Major_Power_Stations.csv")

# columns = X,Y,objectid,featuretype,description,class,name,
#           operationalstatus,owner,generationtype,primaryfueltype,
#           primarysubfueltype,generationmw,generatornumber,locality,state,
#           spatialconfidence,revised,ga_guid,comment_,x_coordinate,y_coordinate

# location as a heuristic

df['location'] = df['name'].map(xymap)

df

# def find_nearest_bus(col):
    # '''
    # Finds the nearest bus_id from buses.csv

    # Parameters
    # ----------
    # col : pandas dataframe
    #     dataframe which includes the x,y coordinates of the power stations and 
    #     x,y coordinates of the buses.

    # Returns
    # -------
    # int
    #     returns the bus_id closest to the power station (heuristic)
    # '''
    # nearestBus = -1
    # xmin,ymin = 100,100

    # totalmin = 100

    # x = col['x_coordinate']
    # y = col['y_coordinate']
    # xbus,ybus = col['bus_xy'].split(',').astype(float)

    # for row_no in range(col.shape(0)):
    #     xt = x-xbus
    #     yt = y-ybus

    #     if (xt**2 + yt**2) < (xmin**2 + ymin**2): # totalmin
    #         xmin,ymin = xt,yt
            
    #         nearestBus = col['bus_id'].loc(row_no)

    # return nearestBus

df.to_csv("../data/grid/generators.csv") # saved as generators.

#### Demand

Demand from substation data + don't need loads on most buses.

#### Converters

DC-DC DC-AC links?

In [1]:
import numpy as np

/var/folders/42/f9_c0v657j907h_6gh8ly4mw0000gn/T/ipykernel_92826/4254518570.py:1: DeprecationWarning: This function is deprecated. Please call randint(10, 50 + 1) instead
  np.random.random_integers(10,50,10)


array([43, 49, 23, 34, 31, 47, 48, 12, 11, 45])